# Deep Learning for HEP events classification
***
* Deep learning is a subfield of machine learning that uses neural networks with multiple layers to automatically learn patterns and relationships from data, enabling tasks like image recognition, natural language processing, and more.
* The basic unit is the artificial neuron (based in a very simple way on the biological neuron).
* The artificial nueron receives inputs from other neurons (or the initial input), combine the information and provide output value(s).
* Here, we will address a classification problem, using tabular data:



## 1. The problem
***

* Event classification is one of the most common and fundamental tasks in the field of high-energy particle physics.
* An **event** corresponds to a proton collision generated in a particle collider, such as the Large Hadron Collider (LHC), and is represented by a set of physical properties measured by the collider's detectors.
* We want to classify events into signal and background categories.
  

## 2. Python packages

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
#import tensorflow_addons as tfa


from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay


## 3. Getting the data
---
* Data is obtained from this [link](https://www.openml.org/d/23512).
* Each event is represented by a set of 28 features, including 21 low-level features corresponding to physics properties measured by the detector, and 7 high-level features derived from the previous ones.
* Some of the event's features:

|Type| Variable  | Description   |
|---| --- | --- |
|low-level|lepton pT |  Momentum of the lepton|
|low-level|lepton eta | Pseudorapidity eta of the lepton|
|low-level|lepton phi | Azimuthal angle phi of the lepton|
|low-level|Missing energy magnitude | Energy not detected|
|| ... | ...|
|high-level|m_jlv| Mass jet ($j$), lepton ($l$, electrons or muons), neutrino $\\nu$|
|high-level|m_bb| Mass quarks $b$|
|high-level|m_wbb| Mass boson $W$ and quarks $b$|
|high-level|m_wwbb|Mass bosons $W$ and quarks $b$|

In [ ]:
#df = pd.read_csv("../data/higgs_reduced.csv")

!gdown https://drive.google.com/uc?id=1z_Y3Jmp8ntiN8egwGSfZl4_D_h5qmRRl


In [ ]:
df = pd.read_csv("higgs_reduced.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df["class"].value_counts()

## 4. Checking for missing values

In [ ]:
def show_null(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    display(missing_data)
show_null(df)

Here, we will simply delete the rows with missing values. However, you can use more sophisticated techniques to perform data imputation.

In [ ]:
df = df.dropna()

## 5. Correlation Matrix

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(12, 10))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, cmap='BrBG', annot = True, fmt=".2f", annot_kws={"size":7})
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':8}, pad=12)

## 6. Pair plots

In [ ]:
sns.set(style="ticks", color_codes=True)
sns.pairplot(df,vars = ['lepton_pT','lepton_eta','lepton_phi'], hue="class")
plt.show()

## 7. Generating training, validation and testing datasets

In [ ]:
y = df["class"]
y = y.astype(int)
X = df.iloc[:,2:-1]

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,
                                                  test_size=0.2, random_state=0)
n_classes = 2

## 8. Building a FFNN using Keras

In [ ]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
import tensorflow  as tf



model = Sequential()
model.add(Dense(300, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(150, activation = "relu"))
model.add(Dense(100, activation = "relu"))
model.add(Dense(50, activation = "relu"))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
metrics_m = [
    tf.keras.metrics.FalseNegatives(name="fn"),
    tf.keras.metrics.FalsePositives(name="fp"),
    tf.keras.metrics.TrueNegatives(name="tn"),
    tf.keras.metrics.TruePositives(name="tp"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
]

model.compile(optimizer=SGD(),
               loss='binary_crossentropy',
               metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

history = model.fit(X_train, y_train, epochs=50,
                    verbose=1,
                    validation_data = (X_val, y_val))

## 9. Performance metrics

In [ ]:
def show_history(history):
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.figure()
    plt.plot(train_loss,'r', label="train")
    plt.plot(val_loss,'g', label="validation")
    plt.legend()
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.show()


def show_metrics(y_pred,th):
    cm = confusion_matrix(y_test, y_pred>th)
    print(cm)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Non-Fraud", "Fraud"])
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    d = classification_report(y_test, y_pred > th,output_dict=True)
    display(pd.DataFrame.from_dict(d))
    return y_pred

In [ ]:
show_history(history)

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob >= 0.5

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score


fpr, tpr, ths = roc_curve(y_test,  y_pred)
auc_ = auc(fpr, tpr)
f1 = f1_score(y_test,  (y_pred>.5))
prec = precision_score(y_test,  (y_pred>.5))
rec = recall_score(y_test,  (y_pred>.5))
acc = accuracy_score(y_test,  (y_pred>.5))
print("F1: %.2f" %f1 , " -- prec: %.2f" %prec, " -- recall: %.2f" %rec, " -- acc: %.2f" %acc)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              )
disp.plot()
plt.show()

In [ ]:
plt.plot(fpr,tpr, label='ROC curve (area = %.2f)' %auc_)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.grid()
plt.show()

In [ ]:
d = classification_report(y_test, y_pred,output_dict=True)

In [ ]:
pd.DataFrame.from_dict(d)

## 10. ANN Score plot
***
Histogram of the ANN scores in the testing set

In [ ]:
#!pip install mplhep

In [ ]:
import mplhep as hep
# Score distribution
f, axs = plt.subplots(1, 1, sharex=True, sharey=True)
h_sig_test, bins_sig_test = np.histogram(y_pred_prob[y_test == 1], bins=30)
h_back_test, bins_back_test = np.histogram(y_pred_prob[y_test == 0], bins=30)
axs.set_title("ANN Classifier")
hep.histplot([h_sig_test, h_back_test], bins_sig_test, ax=axs,label=["Test-S", "Test-B"])
axs.legend()
axs.set_xlabel("Score")
axs.set_ylabel("Number of Events")
plt.tight_layout()
plt.show()

## 11. Using keras_tuner
****
 * To find the best hyperparameters for their machine learning models.

In [ ]:
#!pip install keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
# Define the model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()

    # Input Layer
    model.add(Dense(hp.Int('units_input', min_value=50, max_value=300, step=50),
                    input_shape=(X_train.shape[1],), activation='relu'))

    # Hidden Layers
    for i in range(hp.Int('num_hidden_layers', 1, 4)):
        model.add(Dense(hp.Int(f'units_{i}', min_value=50, max_value=300, step=50),
                        activation='relu'))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# Instantiate the RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of models to try
    executions_per_trial=2,  # Number of times to train each model
    directory='my_dir',  # Directory to save the tuner results
    project_name='random_search_example'
)

# Search for the best model
tuner.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=32)



In [ ]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print(f"Best hyperparameters:\n")
print(f"Input Layer Units: {best_hps.get('units_input')}")
for i in range(best_hps.get('num_hidden_layers')):
    print(f"Hidden Layer {i+1} Units: {best_hps.get(f'units_{i}')}")
print(f"Learning Rate: {best_hps.get('learning_rate')}")

# Build and train the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# Evaluate the model on validation data
val_loss, val_accuracy = best_model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
y_pred_prob_2 = best_model.predict(X_test)
y_pred_2 = y_pred_prob_2 >= 0.5

<div class="alert alert-block alert-success">
<b>Task</b>

<ul>
  <li>Display the performance metrics, confusion matrix, and ANN output (scores) of the best model.</li>
  <li>Modify the tuner to improve the classification performance.</li>
</ul>
    
    
</div>